# 3 mosquito model: invasion probabilities, expected time until invasion, damping ratio, sojourn time

## Abby Barlow, University of Bath
## Ben Adams, University of Bath

Importing required libraries

In [1]:
import numpy as np
import itertools
from scipy.linalg import expm
import pylab as plt
import matplotlib
import sympy as sp

Importing required scripts

In [2]:
import importlib
import Rate_transitions
import Finding_sub_Q
import Lower_block_triangular
import Finding_full_Q
import Entropies
import Hughes_model
import Prob_absorb_to_each
import Time_absorb_wild_states

get_transition = Rate_transitions.get_transition
getQk = Finding_sub_Q.getQk
LBTQ = Lower_block_triangular.LBTQ
getQ = Finding_full_Q.getQ
entropy = Entropies.entropy
F = Hughes_model.F
prob_reach_absorb = Prob_absorb_to_each.prob_reach_absorb
absorb_time_wolb = Time_absorb_wild_states.absorb_time_wolb

# scripts autosave, so no need to re-run code chunk after making changes
%load_ext autoreload
%autoreload 2

Parameter values

In [3]:
# set some parameter values 

K = 3            # reproductive carrying capacity
d1 = 12/100      # wild-type death rate
d2 = 12/100      # Wolbachia death rate
xstar = 2        # imposed steady state (for wild type population)
b1 = round(d1/F(xstar,K),2) # wild-type per capita birth rate
phi = 7/10       # Wolbachia fitness
b2 = b1*phi      # Wolbachia  per capita fitness
v = 10/10        # probability of vertical transmission


# create a dictionary to store all parameter values
params_dict = {'b1': b1,
              'b2': b2,
              'K': K,
              'd1': d1,
              'd2': d2,
              'v': v,
              'phi': phi}

Construct a dictionary of all the state variables

In [4]:
# construct a dictionary that associated an integer index with each possible states, states are stored as an np.array - easier to apply mathematical operations than tuple 
max_pop = 3 # maximum household size
state_dict = {index: np.array((i, j)) for index, (i, j) in enumerate([(i, j) for i in range(max_pop + 1) for j in range(max_pop + 1) if i + j <= max_pop])}

Construct the full transition matrix

In [5]:
# construct a matrix Q for the transition rate q_ij betweeen states i and j
n_states = len(state_dict)  # total number of states

Q = getQ(state_dict,params_dict)
print(Q)

[[-0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.12  -0.288  0.168  0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.24  -0.408  0.168  0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.36  -0.36   0.     0.     0.     0.     0.     0.   ]
 [ 0.12   0.     0.     0.    -0.36   0.     0.     0.24   0.     0.   ]
 [ 0.     0.12   0.     0.     0.12  -0.444  0.084  0.     0.12   0.   ]
 [ 0.     0.     0.12   0.     0.     0.24  -0.36   0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.24   0.     0.    -0.48   0.     0.24 ]
 [ 0.     0.     0.     0.     0.     0.24   0.     0.12  -0.36   0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.36   0.    -0.36 ]]


Constructing individual dictionaries of the communicating state classes and their respective sub-Q matrices

In [6]:
### S1 corresponds to the wild-type-only states, S2 to the Wolbachia-only and S3 the mixed states
state_dict_S1 = {index: np.array((i, 0)) for index, i in enumerate([i for i in range(1,max_pop + 1)])}
state_dict_S2 = {index: np.array((0, i)) for index, i in enumerate([i for i in range(1,max_pop + 1)])}
state_dict_S3 = {index: np.array((i,j)) for index, (i,j) in enumerate([(i, j) for i in range(1,max_pop + 1) for j in range(1,max_pop + 1) if i + j <= max_pop])}

# finding the sub-q matrices and their respective ordered lists of states in the class
# we will use these list to rearrange Q into lower block triangular form
Q1,key_list1 = getQk(state_dict_S1,state_dict,Q,params_dict)
Q2,key_list2 = getQk(state_dict_S2,state_dict,Q,params_dict)
Q3,key_list3 = getQk(state_dict_S3,state_dict,Q,params_dict)

Putting Q in lower block triangular form and constructing the reordered full state dictionary

In [7]:
Q_lower_block_triang, state_dict_relabel = LBTQ(Q,state_dict,state_dict_S1,state_dict_S2,state_dict_S3,max_pop,params_dict)
print('Re-ordered state dictionary is:', state_dict_relabel)
print()
print('Q in lower block triangular form is:', Q_lower_block_triang)

Re-ordered state dictionary is: {0: array([1, 0]), 1: array([2, 0]), 2: array([3, 0]), 3: array([0, 1]), 4: array([0, 2]), 5: array([0, 3]), 6: array([1, 2]), 7: array([1, 1]), 8: array([2, 1])}

Q in lower block triangular form is: [[-0.36   0.24   0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.24  -0.48   0.24   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.36  -0.36   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.    -0.288  0.168  0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.24  -0.408  0.168  0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.36  -0.36   0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.12   0.    -0.36   0.24   0.   ]
 [ 0.12   0.     0.     0.12   0.     0.     0.084 -0.444  0.12 ]
 [ 0.     0.12   0.     0.     0.     0.     0.     0.24  -0.36 ]]


Find the QSD. This is the left eigenvector associated with the over all eigenvalue with minimal magitude normalised to sum to 1. This comes from S1.

In [8]:
# we take the transpose of Q so we obtain the left eigenvector not right
evals, evecs = np.linalg.eig(Q_lower_block_triang.T)  # all eigenvalues and eigenvectors
decay_indx = np.argmax([x for x in evals if x != 0])  # index for over all eigenvalue of minimal magnitude
uvec = evecs[:,decay_indx]                # the corresponding left eigenvector
quasi_stat_dist = uvec/np.sum(uvec)       # normalising to sum to 1
print('The QSD is', quasi_stat_dist)

The QSD is [0.29843788 0.40312424 0.29843788 0.         0.         0.
 0.         0.         0.        ]


Finding the sojourn time

In [9]:
soj_time = 0
for i in range(n_states-1):
    soj_time -= quasi_stat_dist[i]/Q_lower_block_triang[i,i]
print('expected time spent in a state is', soj_time)

expected time spent in a state is 2.497830390671633


Finding the probabilities of reaching the Wolbachia-only state space (successful invasion) conditioning on each transient state

In [10]:
n_transient = len(state_dict_S3) 
prob_reach_wolb = np.zeros(n_transient)
for i in range(max_pop):
    absorb_state = np.array([0,i+1])
    prob_reach_wolb[:] += np.transpose(prob_reach_absorb(state_dict,state_dict_S3,absorb_state,params_dict)[0])[0]
print(prob_reach_wolb)

[0.48051948 0.65367965 0.32034632]


Finding the expected times to invasion originating from eah transient state

In [11]:
absorb_time_wolb(max_pop,np.ones(n_transient),params_dict)

array([4.99883   , 5.22753553, 7.77660778])